> ⚠️ **EDUCATIONAL PURPOSE ONLY**: This lab demonstrates multi-agent coordination patterns using portfolio analysis as an example. This is NOT financial advice and should not be used for actual investment decisions. All portfolio analysis is for educational demonstration of AI agent coordination only.



# Lab 3: Multi-Agent Portfolio Orchestrator

## 📋 Lab Overview

### What You'll Build
In this hands-on lab, you'll create a sophisticated **Multi-Agent Portfolio Orchestrator** that demonstrates advanced AI coordination patterns using the Strands framework. This system combines multiple specialized AI agents to analyze market data, create investment strategies, and enable human oversight.

## 🎯 Learning Objectives
- Understand multi-agent coordination using Strands "agents as tools" pattern  
- Implement human-in-the-loop decision making


## 🏗️ Architecture Overview

This system demonstrates a **Multi-Agent Portfolio Orchestrator** that:
1. **Analyzes** historical market data to create portfolio strategies
2. **Recommends** optimal portfolios using AI analysis  
3. **Enables** human decision-making with natural language overrides

### Agent Architecture
The system consists of:
- **Portfolio Orchestrator**: Coordinates all specialist agents
- **6 Specialist Agents**: Each with focused responsibilities
    - **Stock Data Agent** - Fetches and caches market data with dual CSV storage
    - **Growth Strategy Agent** - Creates high-return, growth-focused portfolios  
    - **Diversified Strategy Agent** - Builds balanced, risk-managed portfolios
    - **Performance Calculator Agent** - Converts abstract returns into concrete $1000 projections
    - **Visualization Agent** - Creates professional charts and comparisons
    - **Validation Agent** - Tests strategies against actual market performance
- **Human-in-the-Loop**: Allow for human intervention, return of control to the human and compare with choices made by the agent.





---

## 🔄 Step 1: Environment Setup

Setting up the Strands framework, AWS Bedrock integration, and importing utility functions that handle the complex portfolio logic. This step prepares all the tools our multi-agent system will need.

In [ ]:
# Install Strands using pip

!pip install -q strands-agents strands-agents-tools matplotlib yfinance

In [ ]:
# Import Strands Agents SDK
from strands import Agent, tool
from strands.models import BedrockModel

# Import standard libraries
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
from datetime import datetime
from typing import Dict, List, Any
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Import utility functions from utils.py
import sys
import os
import importlib
sys.path.append(os.getcwd())

# Import and reload the utils module to get latest changes
import utils
importlib.reload(utils)

# Import all the functions from utils
from utils import (
    load_comprehensive_stock_data_from_csv,
    load_simple_stock_data_from_csv,
    get_stock_data,
    get_stock_analysis,
    create_growth_portfolio,
    create_diversified_portfolio,
    calculate_portfolio_performance,
    visualize_portfolio_allocation,
    visualize_performance_comparison,
    validate_portfolio_performance,
    compare_analysis_accuracy,
    calculate_accuracy_metrics,
    validation_agent,
    # Import safe callback handler (based on Strands samples)
    simple_multi_agent_tracker
)

print("✅ All utility functions imported successfully from utils.py!")
print("🔧 Safe multi-agent tracker imported (with error handling)!")

In [ ]:
# Configure Claude 3.7 Sonnet model
model = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0"
)

print("✅ Claude 3.7 Sonnet configured for multi-agent coordination")


---

## 🔄 Step 2: Agent Creation

Creating 6 specialized agents, each with focused responsibilities. Notice how each agent uses tools from utils.py to keep the code clean and educational. Each agent has a specific role in the portfolio analysis workflow.

### 📊 Stock Data Agent
**Dual-purpose market specialist** - Uses `get_stock_data()` (daily prices + summary) or `get_stock_analysis()` (summary only) based on needs, with intelligent CSV caching for both formats.

In [ ]:
# Import Stock Data Tools from utils.py
print("🔧 Importing Stock Data Tools from utils.py")
print("=" * 50)

# Import the complex functions from utils
from utils import get_stock_data, get_stock_analysis

# Stock Data Agent using both tools
@tool
def stock_data_agent(query: str) -> str:
    """Stock data specialist - chooses appropriate function based on user query"""
    agent = Agent(
        model=model,
        tools=[get_stock_data, get_stock_analysis],
        system_prompt="""You are a Stock Data Specialist with two distinct tools:
        
        - get_stock_data() - Comprehensive: daily prices + summary metrics (annual_return, volatility)
        - get_stock_analysis() - Simple: summary metrics only (return_pct, volatility_pct)
        
        Choose based on needs:
        - Use get_stock_data() when daily price data is needed
        - Use get_stock_analysis() for quick portfolio analysis
        
        Both automatically use CSV caching. Present analysis clearly showing what data was used.
        Keep it simple and educational."""
    )
    return str(agent(query))

print("✅ Stock data functions imported from utils.py:")
print("  🛠️ get_stock_data() - Daily prices + summary (annual_return, volatility)")
print("  📊 get_stock_analysis() - Summary only (return_pct, volatility_pct)")
print("  📈 stock_data_agent() - Chooses appropriate tool")


### 🚀 Growth Strategy Agent
**High-growth portfolio specialist** - Uses `create_growth_portfolio()` to build portfolios focused on maximum return potential with higher risk tolerance.


In [ ]:
# Create Growth Strategy Agent (Portfolio Creation Logic in utils.py)
print("🚀 Creating Growth Strategy Agent")
print("=" * 40)

# Streamlined Growth Strategy Agent - business logic is in utils.py
@tool
def growth_strategy_agent(query: str) -> str:
    """Streamlined growth strategy specialist - uses portfolio creation functions from utils.py"""
    agent = Agent(
        model=model,
        tools=[create_growth_portfolio],
        system_prompt="""You are a Growth Strategy Specialist focused on high-growth portfolios.

        Your approach:
        1. Use create_growth_portfolio() with sophisticated allocation methods
        2. Choose allocation method based on strategy:
           - "performance_weighted" - Weight by relative returns (default, more aggressive)
           - "risk_adjusted" - Weight by risk-adjusted returns (more balanced growth)
           - "equal_weight" - Simple equal allocation (basic approach)
        3. Present clear analysis and reasoning for growth-focused selections
        
        Available tool:
        - create_growth_portfolio(allocation_method="performance_weighted") - Creates growth portfolios with smart allocation
        
        Advanced usage:
        - Use allocation_method parameter to control weighting strategy
        - "performance_weighted" gives higher allocations to higher-return stocks
        - "risk_adjusted" balances returns with volatility for smarter growth
        
        Format your final recommendation as:
        PORTFOLIO: {allocation dictionary with varied percentages}
        STRATEGY: Growth ({allocation_method})
        EXPECTED RETURN: X%
        RISK LEVEL: High/Moderate
        ALLOCATION METHOD: {chosen method and reasoning}
        
        Focus on creating dynamic allocations, not just 25% equal weights."""
    )
    return str(agent(query))

print("✅ Growth Strategy Agent created!")


### ⚖️ Diversified Strategy Agent
**Balanced portfolio specialist** - Uses `create_diversified_portfolio()` and risk-adjusted metrics to build sector-diversified portfolios with moderate risk.

In [ ]:
# Create Diversified Strategy Agent (Portfolio Creation Logic in utils.py)
print("⚖️ Creating Diversified Strategy Agent")
print("=" * 45)

# Streamlined Diversified Strategy Agent - business logic is in utils.py
@tool
def diversified_strategy_agent(query: str) -> str:
    """Streamlined diversified strategy specialist - uses portfolio creation functions from utils.py"""
    agent = Agent(
        model=model,
        tools=[create_diversified_portfolio],
        system_prompt="""You are a Diversified Strategy Specialist focused on balanced portfolios.

        Your approach:
        1. Use create_diversified_portfolio() to build risk-balanced portfolios
        2. Focus on risk-adjusted returns and sector diversification
        3. Present clear analysis and reasoning for balanced selections
        
        Available tool:
        - create_diversified_portfolio() - Creates balanced portfolios (logic in utils.py)
        
        Format your final recommendation as:
        PORTFOLIO: {allocation dictionary}
        STRATEGY: Diversified
        EXPECTED RETURN: X%
        RISK LEVEL: Low/Moderate
        SECTORS: X different sectors
        
        Keep it simple and educational - focus on the strategy, not the implementation."""
    )
    return str(agent(query))

print("✅ Diversified Strategy Agent created!")


### 💰 Performance Calculator Agent
**Investment analysis specialist** - Uses `calculate_portfolio_performance()` to transform abstract returns into concrete $1000 investment projections and risk assessments.

In [ ]:
# Create Performance Calculator Agent (Calculation Logic in utils.py)
print("💰 Creating Performance Calculator Agent")
print("=" * 45)

# Streamlined Performance Calculator Agent - business logic is in utils.py
@tool
def performance_calculator_agent(query: str) -> str:
    """Streamlined performance calculator specialist - uses calculation functions from utils.py"""
    agent = Agent(
        model=model,
        tools=[calculate_portfolio_performance, create_growth_portfolio, create_diversified_portfolio],
        system_prompt="""You are a Performance Calculator Specialist focused on concrete investment analysis.

        Your approach:
        1. Use portfolio creation tools to get strategies for comparison
        2. Use calculate_portfolio_performance() to analyze returns and make concrete projections
        3. Present clear comparison between strategies with concrete $1000 projections
        
        Available tools:
        - calculate_portfolio_performance() - Calculates performance metrics (logic in utils.py)
        - create_growth_portfolio() - Get growth portfolio for analysis
        - create_diversified_portfolio() - Get diversified portfolio for analysis
        
        Format your final summary as:
        PERFORMANCE ANALYSIS:
        Strategy 1: $X final value (Y% return, Z risk)
        Strategy 2: $X final value (Y% return, Z risk)
        
        WINNER: [Strategy] - [Reasoning]
        
        Make abstract returns tangible through concrete dollar examples."""
    )
    return str(agent(query))

print("✅ Performance Calculator Agent created!")


### 🎨 Visualization Agent
**Chart and visualization specialist** - Uses `visualize_portfolio_allocation()` and `visualize_performance_comparison()` to create professional charts that make portfolio data visually accessible and easy to interpret.


In [ ]:
# Create Visualization Agent (Visualization Logic in utils.py)
print("🎨 Creating Visualization Agent")
print("=" * 35)

# Streamlined Visualization Agent - visualization logic is in utils.py
@tool
def visualization_agent(query: str) -> str:
    """Streamlined visualization specialist - uses visualization functions from utils.py"""
    agent = Agent(
        model=model,
        tools=[visualize_portfolio_allocation, visualize_performance_comparison, create_growth_portfolio, create_diversified_portfolio],
        system_prompt="""You are a Visualization Specialist focused on creating clear, informative charts.

        Your approach:
        1. Use portfolio creation tools to get data for visualization
        2. Use visualize_portfolio_allocation() for pie charts showing allocation breakdowns
        3. Use visualize_performance_comparison() for bar charts showing performance metrics
        4. Create clear, professional visualizations that enhance understanding
        
        Available tools:
        - visualize_portfolio_allocation() - Creates pie charts (logic in utils.py)
        - visualize_performance_comparison() - Creates bar charts (logic in utils.py)
        - create_growth_portfolio() - Get growth portfolio data for visualization
        - create_diversified_portfolio() - Get diversified portfolio data for visualization
        
        Focus on making data visually accessible and easy to interpret. Always provide context for what the charts show."""
    )
    return str(agent(query))

print("✅ Visualization Agent created!")
print("  🎯 Focused on multi-agent coordination pattern")
print("  📊 Visualization logic moved to utils.py")
print("  🔧 Much cleaner and more educational")

### 🔍 Validation Agent
**Portfolio validation specialist Agent** - Uses `validate_portfolio_performance()`, `compare_analysis_accuracy()`, and `calculate_accuracy_metrics()` from utils.py to test portfolios against actual market data.

---

## 🔄 Step 3: Multi-Agent Coordination

The Portfolio Orchestrator coordinates all specialist agents to analyze market data, create strategies, and provide visual comparisons. Watch how the master agent delegates tasks to specialists and synthesizes their outputs into a coherent recommendation.

## 🚀 Multi-Agent Workflow Execution

Let's run the complete multi-agent portfolio orchestration workflow!

In [ ]:
# Execute the multi-agent workflow
user_request = """
Create an optimal investment portfolio for me using your multi-agent system.

I want you to:
1. Analyze current market data using your specialist agents
2. Create both growth and diversified portfolio strategies
3. Show me the AI recommendation with visual comparisons
4. Calculate concrete $1000 investment projections
5. Allow me to review and potentially override the recommendation

Please walk me through each step of the multi-agent coordination process.
"""


# Portfolio Orchestrator with all specialist agents as tools
portfolio_orchestrator = Agent(
    model=model,
    system_prompt="""
    You are the Portfolio Orchestrator, the master agent coordinating specialist agents for portfolio creation.
    
    Your streamlined workflow:
    1. Use stock_data_agent to ensure both CSV caches are populated with market data for year 2024
    2. Use growth_strategy_agent to create high-growth portfolio
    3. Use diversified_strategy_agent to create balanced portfolio  
    4. Use performance_calculator_agent to calculate concrete $1000 investment projections
    5. Use visualization_agent to create clear charts showing portfolio comparisons
    6. Compare strategies and present clear recommendation with reasoning
    7. For validation with validation_agent, use market data for year 2025 
    8. Allow human review and potential override with natural language
    
    Available specialist agents:
    - stock_data_agent: Ensures both comprehensive and simple data caches are available
    - growth_strategy_agent: Creates high-growth portfolios using cached data
    - diversified_strategy_agent: Creates balanced portfolios using cached data
    - performance_calculator_agent: Calculates concrete investment returns and comparisons
    - visualization_agent: Creates professional charts for portfolio analysis
    
    Key principles:
    - All agents use efficient CSV caching for fast performance
    - Present concrete $1000 investment examples to make returns tangible
    - Use visualizations to enhance understanding of portfolio differences
    - Provide clear reasoning for recommendations
    - Allow human decision-making with natural language input
    
    """,    
    tools=[
        stock_data_agent,
        growth_strategy_agent, 
        diversified_strategy_agent,
        performance_calculator_agent,
        visualization_agent,
        validation_agent
    ],
)

print("🎬 Starting Multi-Agent Portfolio Orchestration...")
print("=" * 60)

# Run the portfolio orchestrator
result = portfolio_orchestrator(user_request)
print(result)


---

## 🔄 Step 4: Human-in-the-Loop Decision

This is where you review the AI recommendation and either accept it or override with your own requirements using natural language. The system demonstrates transparent AI where humans retain final decision control.

## 👤 Human-in-the-Loop Decision Point

This is where you can accept the AI recommendation or override with custom requirements.

In [ ]:
# Human decision point - customize this based on your preference
human_decision = """
OPTION 1 - Accept AI Recommendation:
\"Accept the AI recommendation\"

OPTION 2 - Override with Custom Requirements (examples):
\"Override: I want more technology focus\"
\"Override: Make it more conservative\"
\"Override: Add more dividend stocks\"
\"Override: Reduce risk but keep some growth\"

Choose your decision below:
"""

print(human_decision)

# Uncomment and modify one of these lines based on your choice:
# user_decision = \"Accept the AI recommendation\"
# user_decision = \"Override: I want more technology focus\"
# user_decision = \"Override: Make it more conservative\"

user_decision = "I want more technology focus"  # Default for demo

print(f"\n🎯 Your Decision: {user_decision}")
print("\n📊 Processing your decision and creating final portfolio...")

final_result = portfolio_orchestrator(f"""
The human has made their decision: \"{user_decision}\"

Please:
1. Process this decision appropriately
2. Create or modify the portfolio based on their choice
3. Use the validation agent to test against 2025 actual market data
4. Create final visualizations showing the results
5. Provide a summary of the complete multi-agent workflow
""")

print(final_result)

## 🎓 Learning Summary

### What We Learned

1. **Multi-Agent Coordination**: How to coordinate specialist agents using the Strands \"agents as tools\" pattern

2. **Human-in-the-Loop**: How to implement transparent AI recommendations with human oversight

3. **Portfolio Analysis**: How to analyze historical market data to create investment strategies

4. **Market Validation**: How to test analysis accuracy using actual market performance

5. **Agent Specialization**: How each agent has focused responsibilities:
   - Stock Data Agent: Market data and metrics
   - Growth Strategy Agent: High-growth portfolios
   - Diversified Strategy Agent: Balanced portfolios
   - Performance Agent: Concrete $1000 projections
   - Visualization Agent: Charts and comparisons
   - Validation Agent: Market reality testing

### Key Insights

- **Historical Analysis ≠ Future Performance**: Our validation shows the limitations of historical analysis
- **Human Judgment Matters**: AI provides intelligence, but humans retain decision control
- **Transparency is Critical**: Users see AI reasoning before making decisions
- **Concrete Examples Help**: $1000 projections make abstract returns tangible
- **Agent Coordination Works**: Complex workflows through intelligent agent coordination

This demonstrates responsible AI deployment where AI provides intelligence while humans retain control over final decisions.